In [2]:
from collections import OrderedDict

# imports
import numpy as np
import shap
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from pandas.core.interchange.dataframe_protocol import DataFrame
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
#from sympy.abc import lamda
#from sympy.strategies.core import switch
import threading
from concurrent.futures import ThreadPoolExecutor
import indicators
import importlib
importlib.reload(indicators)
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from collections import OrderedDict

In [2]:
stocks = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "META", "TSLA", "NVDA", "NFLX", "AMD", "INTC",
    "JPM", "GS", "BAC", "C", "WFC", "V", "MA", "AXP", "BRK-B",
    "UNH", "JNJ", "PFE", "LLY", "ABBV", "TMO", "DHR", "BMY", "GILD",
    "XOM", "CVX", "COP", "OXY", "SLB", "HAL", "BP", "SHEL", "EOG",
    "WMT", "COST", "HD", "LOW", "MCD", "SBUX", "TGT", "NKE", "PG", "KO",
    "ADBE", "CRM", "ORCL", "QCOM", "AVGO", "TXN", "INTU", "CSCO", "IBM", "MU",
    "PANW", "NOW", "CDNS", "ANET", "LRCX", "SNPS", "ACN", "FTNT", "MSI", "ZM",
    "BLK", "TFC", "USB", "BK", "SCHW", "SPGI", "MS", "ICE", "PGR", "AIG",
    "CB", "MMC", "MET", "ALL", "PRU", "CME", "COF", "DFS", "FITB", "MTB",
    "ABT", "MRK", "ZBH", "ISRG", "MDT", "CVS", "CI", "REGN", "VRTX", "SYK",
    "PSX", "MPC", "VLO", "KMI", "WMB", "CTRA", "DVN", "HES", "FANG",
    "LIN", "APD", "ECL", "SHW", "NEM", "FCX", "DD", "ALB", "LYB", "MLM",
    "PEP", "CL", "KMB", "EL", "MNST", "KR", "DG", "DLTR", "GIS", "MDLZ",
    "TSCO", "ROST", "TJX", "YUM", "WBA", "PM", "MO", "UL", "HSY", "HRL",
    "CAT", "DE", "HON", "LMT", "RTX", "BA", "GE", "NOC", "ETN", "EMR",
    "UNP", "NSC", "FDX", "UPS", "CSX", "WM", "EXC", "DUK", "NEE",
    "PLD", "AMT", "CCI", "EQIX", "SPG", "PSA", "O", "DLR", "VTR", "ARE"
]

# Alternatively, use stocks2 if you want to
stocks2 = [
    "ADBE", "CRM", "ORCL", "SAP", "NOW", "SHOP", "SQ", "ZM", "CRWD", "DDOG",
    "TXN", "QCOM", "AVGO", "MU", "LRCX", "KLAC", "NXPI", "ADI", "MRVL", "SWKS",
    "PYPL", "INTU", "FISV", "ADP", "VEEV", "TEAM", "WDAY", "ZS", "OKTA", "MDB",
    "T", "VZ", "TMUS", "CHTR", "CMCSA", "DIS", "ROKU", "LYV", "TTWO", "ATVI",
    "PEP", "KMB", "CL", "HSY", "MDLZ", "GIS", "MO", "PM", "EL", "STZ"
]

real_stock_portfolio_3 = [
    "AAPL", "ABNB", "ACN", "ALAB", "AMD", "AMZN", "ANET", "AOSL", "APP",
    "ASAN", "ASML", "AVGO", "BAH", "BITO", "BWXT", "CLS", "COHR", "COIN", "COST",
    "COWG", "CPRX", "CRDO", "CRM", "CRWV", "DAVE", "DELL", "DKNG", "DOCS",
    "DXPE", "EPD", "FBTC", "FVRR", "GOOG", "GRNY", "HOOD", "IHAK", "INTA", "IONQ",
    "JPM", "LITE", "LQDT", "LUNR", "META", "MRVL", "MSFT", "MU", "NBIS", "NEE",
    "NFLX", "NLR", "NNE", "NUTX", "NVDA", "NVDY", "NVO", "OUST", "OXY", "PANW",
    "PEP", "PLD", "PLTR", "PYPL", "QCOM", "QTUM", "RBRK", "RDDT", "RDNT", "REAL",
    "RGTI", "S", "SAIC", "SCHD", "SEZL", "SKYW", "SMCI", "SMTC", "SNOW", "SOXL",
    "SYM", "TEAM", "TEM", "TOST", "TSM", "U", "UBER", "UPST", "URA", "VIST",
    "VRT", "WMT", "WRD", "XYZ", "HIMS", "OSCR"
]

# One of my ideas was to take some stocks and put them into sectors to train their own models. Didn't really work out.
"""sectors_tickers = pd.read_excel("data/100_stocks_sectors.xlsx")
healthcare = tuple(sectors_tickers["Healthcare"].to_list())
consumer_discretionary = tuple(sectors_tickers["Consumer Discretionary"].to_list())
consumer_staples = tuple(sectors_tickers["Consumer Staples"].to_list())
communication = tuple(sectors_tickers["Communication"].to_list())
energy = tuple(sectors_tickers["Energy"].to_list())
financials = tuple(sectors_tickers["Financials"].to_list())
industrials = tuple(sectors_tickers["Industrials"].to_list())
technology = tuple(sectors_tickers["Tech"].to_list())
utilities = tuple(sectors_tickers["Utilities"].to_list())
real_estate = tuple(sectors_tickers["Real Estate"].to_list())
materials = tuple(sectors_tickers["Materials"].to_list())
overall_tickers = tuple(list(healthcare) + list(consumer_discretionary) + list(consumer_staples) + list(communication) + list(energy) + list(financials) + list(industrials) + list(technology) + list(utilities) + list(real_estate) + list(materials))


sectors_dict = {
    healthcare: "Healthcare",
    consumer_discretionary: "ConsumerDiscretionary",
    consumer_staples: "ConsumerStaples",
    communication: "Communication",
    energy: "Energy",
    financials: "Financials",
    industrials: "Industrials",
    technology: "Technology",
    utilities: "Utilities",
    real_estate: "RealEstate",
    materials: "Materials"
}"""


FileNotFoundError: [Errno 2] No such file or directory: 'data/100_stocks_sectors.xlsx'

In [ ]:
# Uncomment this and/or the next cell if you want to refresh the dataset(s)
#x = yf.download(stocks, start="2015-01-01", end="2025-04-15", interval="1wk")

In [ ]:
"""
sp500 = yf.download("^GSPC", start="2015-01-01", interval="1wk")

sp500.columns = ["_".join(col) if isinstance(col, tuple) else col for col in sp500.columns]

sp500 = sp500.loc[:, ~sp500.columns.isin(["level_0", "index"])]
sp500.to_csv("SP500.csv")
"""

In [ ]:
def download_and_fix_yfinance_data(stocks, csv_filename): # Takes raw yfinance dataframe and fixes it if it has that weird date syncing problem
    data = {}

    for t in stocks:
        df = yf.download(t, interval='1d', start='2015-01-01', auto_adjust=True)
        df = df.resample('W-FRI').agg({ # everything needs to be on the friday interval
          ('Open', t): 'first',
          ('High', t): 'max',
          ('Low', t): 'min',
          ('Close', t): 'last',
          ('Volume', t): 'sum'
        })
        df.name = t
        data[t] = df

    # Combine and align on the same date index
    combined = pd.concat(data.values(), axis=1, join='outer')
    combined = combined.ffill()  # or .bfill(), depending on your use case

    non_nan_counts = combined.count()
    cols_to_drop = non_nan_counts[non_nan_counts < 20].index
    y = combined.drop(columns=cols_to_drop)

    stockData = y.copy()

    # Rename columns, joining multi-level column names into a single string with "_".
    stockData.columns = ["_".join(col) if isinstance(col, tuple) else col for col in stockData.columns]

    # Remove unnecessary columns such as "level_0" or "index" that may have been carried over.
    stockData = stockData.loc[:, ~stockData.columns.isin(["level_0", "index"])]
    stockData.to_csv(csv_filename + ".csv")

#for sector_list, sector_name in sectors_dict.items():
#  download_and_fix_yfinance_data(sector_list, sector_name)


In [ ]:
def download_and_fix_sp500():
    y = yf.download("^GSPC", start="2015-01-01", interval="1d")
    y = y.resample('W-FRI').agg({
          ('Open', "^GSPC"): 'first',
          ('High', "^GSPC"): 'max',
          ('Low', "^GSPC"): 'min',
          ('Close', "^GSPC"): 'last',
          ('Volume', "^GSPC"): 'sum'
        })
    sp500 = y.copy()

    sp500.columns = ["_".join(col) if isinstance(col, tuple) else col for col in sp500.columns]

    sp500 = sp500.loc[:, ~sp500.columns.isin(["level_0", "index"])]
    sp500.to_csv("SP500.csv")

In [ ]:
def extract_ticker_dataframe(csv_filepath: str, ticker: str) -> pd.DataFrame:
    """
    Reads a multi-ticker CSV file (like one from yfinance) and isolates the data
    for the specified ticker. This updated version assumes that the CSV contains the
    dates as the index (first column), so we use that as the Date information.

    The CSV is expected to have a two-row header:
      - The first row contains field names (e.g., "Open", "High", etc.).
      - The second row contains the ticker symbols for each column.

    The returned DataFrame's columns will be ordered as needed for Backtrader:
      Date, Open, High, Low, Close, Volume.
    """
    # Use the first column as the index so that the dates are read from the CSV.
    df = pd.read_csv(csv_filepath)#, header=[0, 1], index_col=0)

    # Convert the index to datetime in case it's not already
    df.index = pd.to_datetime(df["Date"], errors="coerce")
    df = df.drop("Date", axis=1)

    # Identify all columns for the specified ticker (matching on the lower level).
    ticker_cols = [col for col in df.columns if ticker == str(col).strip().split("_")[-1]]
    if not ticker_cols:
        raise ValueError(f"Ticker '{ticker}' not found in the CSV file.")

    # Extract the ticker's columns
    df_ticker = df.loc[:, ticker_cols].copy()
    #df_ticker.columns = [col[0].strip() for col in df_ticker.columns]

    # Removing ticker name from column names
    for col in df_ticker.columns:
        df_ticker.rename(columns={col: str(col).split("_")[0]}, inplace=True)

    desired_order = ["Open", "High", "Low", "Close", "Volume"]

    available_order = [col for col in desired_order if col in df_ticker.columns]
    df_ticker = df_ticker[available_order]

    df_ticker = df_ticker.reset_index().rename(columns={"index": "Date"}).set_index("Date")

    return df_ticker

In [ ]:
# If you uncommented the yf.download cells, uncomment these as well. One is for the stocks and the other is for the sp500
'''
stockData = x.copy()

# Rename columns, joining multi-level column names into a single string with "_".
stockData.columns = ["_".join(col) if isinstance(col, tuple) else col for col in stockData.columns]

# Remove unnecessary columns such as "level_0" or "index" that may have been carried over.
stockData = stockData.loc[:, ~stockData.columns.isin(["level_0", "index"])]
stockData.to_csv("50stocks.csv")
'''


'''
sp500 = y.copy()

sp500.columns = ["_".join(col) if isinstance(col, tuple) else col for col in sp500.columns]

sp500 = sp500.loc[:, ~sp500.columns.isin(["level_0", "index"])]
sp500.to_csv("SP500.csv")'''

In [ ]:
# 1. Feature engineering: Hamilton Regime Switching Model
sp500 = pd.read_csv("data/SP500.csv")
sp500.set_index("Date", inplace=True)
sp500.index = pd.to_datetime(sp500.index)
sp500['Log_Returns'] = np.log(sp500['Close_^GSPC'] / sp500['Close_^GSPC'].shift(1))
sp500['Returns_6wk'] = sp500['Close_^GSPC'].pct_change(6)
sp500 = sp500.dropna()

def classify_regimes(sp500: pd.DataFrame) -> pd.DataFrame:
    model = MarkovRegression(sp500['Log_Returns'], k_regimes=2, trend='c', switching_variance=True)
    result = model.fit()
    #print(result.summary())
    smoothed_probs = result.smoothed_marginal_probabilities
    sp500['Regime'] = smoothed_probs.idxmax(axis=1)
    sp500['Bull_Prob'] = smoothed_probs[0]

    """if show_regimes:
        plt.plot(sp500.index, smoothed_probs[0], label="Probability of Bull Market")
        plt.fill_between(sp500.index, 0, 1, where=sp500['Bull_Prob'] > 0.5, color='green', alpha=0.3)
        plt.fill_between(sp500.index, 0, 1, where=sp500['Bull_Prob'] <= 0.5, color='red', alpha=0.3)
        plt.legend()
        plt.title("Bull vs. Bear Market Probability")
        plt.show()"""

    return sp500["Bull_Prob"].to_frame()
    # 0 -> Bull, 1 -> Bear

def compute_beta(asset_returns, market_returns, window=20):
    cov = asset_returns.rolling(window).cov(market_returns)
    var = market_returns.rolling(window).var()
    return cov / var


In [ ]:
# Compute rolling portfolio weights using a lookback period (e.g., 52 weeks)
def compute_rolling_portfolio_weights(data, lookback_window=52):
    """
    Computes portfolio weights for each date using historical data up to that date.

    Args:
        data (pd.DataFrame): DataFrame with dates as index and stocks as columns.
        lookback_window (int): Number of days to look back for the optimization.

    Returns:
        pd.DataFrame: A DataFrame with dates as index and stocks as columns, containing weights.
    """
    weights_list = []
    dates = []
    for date in data.index[lookback_window:]:
        window_data = data.loc[:date].tail(lookback_window)
        mu = expected_returns.mean_historical_return(window_data)
        S = risk_models.sample_cov(window_data)
        ef = EfficientFrontier(mu, S)
        try:
            ef.max_sharpe()
            clean_weights = ef.clean_weights()
        except Exception as e:
            # In case optimization fails, assign zero weights.
            clean_weights = {stock: 0 for stock in data.columns}
        weights_list.append(clean_weights)
        dates.append(date)
    weights_df = pd.DataFrame(weights_list, index=dates)
    return weights_df

In [ ]:
# 2. Feature engineering: Technical indicators, fundamentals
# SMA, RSI, MACD, etc. etc. basically, seeing which indicator sticks
def create_stock_features(stocks, stock_data_filename) -> pd.DataFrame:
    feature_rows = []
    regime_df = classify_regimes(sp500)

    for stock in stocks:
        # will be filled with indicators for one stock
        try:
            prices = extract_ticker_dataframe(stock_data_filename, stock)
        except ValueError:
            continue
        features = pd.DataFrame(index=prices.index)

        # Simple Moving Avg Comparison (5 vs 20)
        sma = indicators.sma_strategy(prices["Close"].to_frame(), 5, 20)
        features["SMA_5v20"] = sma["signal_raw"]

        # Volume
        features["Volume"] = prices["Volume"]

        # Relative Strength Index (RSI)
        rsi = indicators.rsi_strategy(prices["Close"].to_frame())
        features["RSI"] = rsi["rsi"]

        # Moving Average Convergence Divergence (MACD)
        macd = indicators.macd_strategy(prices["Close"].to_frame())
        features["MACD"] = macd["signal_raw"] # only using signal, not other columns, since I don't want to have weird "fitting" of the model to the components of the macd signal

        # Bollinger Bands
        bands = indicators.bollinger_strategy(prices["Close"].to_frame())
        features["Bollinger_Bands"] = bands["signal_ternary"]

        # Average True Range (ATR)
        atr = indicators.atr_indicator(prices[["High", "Low", "Close"]])
        features["ATR"] = atr["signal"]

        # Stochastic Oscillator Strategy
        stochastic = indicators.stochastic_strategy(prices[["High", "Low", "Close"]])
        features["Stochastic"] = stochastic["signal"]

        # OBV (On-Balance Volume)
        obv = indicators.obv_strategy(prices[["Close", "Volume"]])
        features["OBV"] = obv["obv"]

        # ADX (Average Directional Index)
        adx = indicators.adx_strategy(prices[["High", "Low", "Close"]])
        features["ADX"] = adx["adx"]

        # Aroon Indicator
        aroon = indicators.aroon_strategy(prices[["High", "Low"]])
        features["Aroon"] = aroon["aroon_oscillator"]

        # Returns features. Overall 4 week percent change, split into 3 week period, 1 week lag and 1 week period, 0 week lag
        features["Returns-3wk-1wklag"] = prices["Close"].pct_change(periods=3).shift(1)
        features["Returns-1wk-0wklag"] = prices["Close"].pct_change()

        # Other technical indicators and fundamentals?

        # rolling return (2 week window)
        #prices["Returns-2wk"] = prices["Close"].pct_change(periods=2)
        #features["Returns-2wk"] = prices["Returns-2wk"]

        # future return (for labels)
        features['Returns-future-1wk'] = prices['Close'].pct_change(periods=1).shift(-1)
        features['Returns-future-2wk'] = prices['Close'].pct_change(periods=2).shift(-2)

        features["Bull_Probability"] = regime_df["Bull_Prob"]

        # Volatility features
        features['SP500-Log-Returns'] = sp500['Log_Returns']
        features['SP500-Returns'] = sp500['Returns_6wk']

        prices['Log_Returns'] = np.log(prices['Close'] / prices['Close'].shift(1))

        prices['Volatility_4wk'] = prices['Log_Returns'].rolling(window=4).std()
        prices['Volatility_12wk'] = prices['Log_Returns'].rolling(window=12).std()
        features['Volatility_Spike'] = prices['Volatility_4wk'] / prices['Volatility_12wk']

        features['Beta_26wk'] = compute_beta(prices['Log_Returns'], sp500['Log_Returns'])

        # Time information
        features['Week-of-year'] = prices.index.isocalendar().week
        features['Month-of-year'] = prices.index.month
        features['Quarter'] = prices.index.quarter

        # Year of presidential term
        features['Presidential-year'] = prices.index.to_series().apply(presidential_term_year)


        features["Stock"] = stock
        features = features.reset_index().rename(columns={"index": "Date"})
        feature_rows.append(features)

    # Combine data for all stocks into one dataframe
    features_long = pd.concat(feature_rows, ignore_index=True).set_index("Date").dropna()
    return features_long

def presidential_term_year(date):
    # Start years of presidential terms
    election_years = [2000, 2004, 2008, 2012, 2016, 2020, 2024]
    year = date.year
    for start in reversed(election_years):
        if year >= start:
            return (year - start) + 1  # 1 to 4
    return np.nan

overall_df = create_stock_features(overall_tickers, "data/training_set.csv")
"""communication_df = create_stock_features(communication, "data/Communication.csv")
consumer_staples_df = create_stock_features(consumer_staples, "data/ConsumerStaples.csv")
consumer_discretionary_df = create_stock_features(consumer_discretionary, "data/ConsumerDiscretionary.csv")
energy_df = create_stock_features(energy, "data/Energy.csv")
financials_df = create_stock_features(financials, "data/Financials.csv")
healthcare_df = create_stock_features(healthcare, "data/Healthcare.csv")
industrials_df = create_stock_features(industrials, "data/Industrials.csv")
real_estate_df = create_stock_features(real_estate, "data/RealEstate.csv")
technology_df = create_stock_features(technology, "data/Technology.csv")
utilities_df = create_stock_features(utilities, "data/Utilities.csv")
materials_df = create_stock_features(materials, "data/Materials.csv")"""

In [ ]:
def buy_low_sell_high_strategy(row):
    returns_prev_week = row['Returns-1wk-0wklag']
    if returns_prev_week < 0:
        return 1 # Buy
    else:
        return 0 # Sell

def buy_low_sell_high_model(df):
    df['Signal'] = df.apply(buy_low_sell_high_strategy, axis=1)
    df = df.sort_values(by='Date')
    return df

df = overall_df.copy()
df = buy_low_sell_high_model(df) # df now contains the naive model's actions, includes features, though. When we use backtrader, we will probably get rid of those. 

In [ ]:
def label_signal2(row, buy_thresh=0.01, sell_thresh=-0.015, vol_thresh=0.02):
    r1 = row['Returns-future-1wk']
    r2 = row['Returns-future-2wk']
    if r1 < -0.01 and r2 < -0.015:
        return 0  # Sell
    elif r1 > 0.01 and r2 > 0.015:
        return 1  # Buy
    else:
        return 2  # Hold

def label_signal(row):
    r1 = row['Returns-future-1wk']
    r2 = row['Returns-future-2wk']
    if r1 < -0.02 and r2 < -0.03:
        return 0  # Strong Sell
    elif r1 < -0.01 and r2 < -0.02:
        return 1  # Weak Sell
    elif r1 > 0.02 and r2 > 0.03:
        return 4  # Strong Buy
    elif r1 > 0.01 and r2 > 0.02:
        return 3  # Weak Buy
    else:
        return 2  # Hold


def train_val_test_split(df):
    df['Signal'] = df.apply(label_signal2, axis=1)
    df = df.dropna(subset=['Returns-future-1wk', 'Returns-future-2wk', 'Signal']) # dropping rows with no signal

    # Sort by date?
    df = df.sort_values(by='Date')

    # 70% train, 15% val, 15% test
    split_1 = int(len(df) * 0.7)
    split_2 = int(len(df) * 0.85)

    train = df.iloc[:split_1]
    val = df.iloc[split_1:split_2]
    test = df.iloc[split_2:]

    return train, val, test



In [ ]:
def train_model(sets: tuple):
    train = sets[0]
    features = train.drop(['Stock', 'Signal', 'Returns-future-1wk', 'Returns-future-2wk'], axis=1).columns.values
    X_train = train[features]
    y_train = train['Signal']

    # balanced class weights to balance any over-representation
    classes = np.unique(y_train)
    class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
    class_weight_dict = dict(zip(classes, class_weights))
    print("Class Weights:", class_weight_dict)

    sample_weights = y_train.map(class_weight_dict)

    # Train the model
    model = XGBClassifier(
        objective='multi:softprob',  # for multi-class
        num_class=len(classes),
        eval_metric='mlogloss',
        tree_method='hist',
        subsample=0.6,
        colsample_bytree=0.9,
        reg_lambda=5,
        reg_alpha=5,
        booster='gbtree',
        learning_rate=0.01,
        gamma=0.5,
        min_child_weight=5,
        n_estimators=200,
        use_label_encoder=False,
        random_state=42,
        max_depth = 12,
    )
    model.fit(X_train, y_train, sample_weight=sample_weights)

    return model

def predict(model: XGBClassifier, sets: tuple, val_or_test: str):
    val = sets[1]
    test = sets[2]
    features = val.drop(['Stock', 'Signal', 'Returns-future-1wk', 'Returns-future-2wk'], axis=1).columns.values

    if val_or_test == 'val':
        x = val[features]
        y = val['Signal']
    else:
        x = test[features]
        y = test['Signal']

    y_pred = model.predict(x)
    return classification_report(y, y_pred, output_dict=True)



In [ ]:
overall_sets = train_val_test_split(overall_df)
"""communication_sets = train_val_test_split(communication_df)
energy_sets = train_val_test_split(energy_df)
consumer_discretionary_sets = train_val_test_split(consumer_discretionary_df)
consumer_staples_sets = train_val_test_split(consumer_staples_df)
financials_sets = train_val_test_split(financials_df)
healthcare_sets = train_val_test_split(healthcare_df)
industrials_sets = train_val_test_split(industrials_df)
real_estate_sets = train_val_test_split(real_estate_df)
technology_sets = train_val_test_split(technology_df)
materials_sets = train_val_test_split(materials_df)
utilities_sets = train_val_test_split(utilities_df)"""

overall_model = train_model(overall_sets)
"""communication_model = train_model(communication_sets)
energy_model = train_model(energy_sets)
consumer_discretionary_model = train_model(consumer_discretionary_sets)
consumer_staples_model = train_model(consumer_staples_sets)
financials_model = train_model(financials_sets)
healthcare_model = train_model(healthcare_sets)
industrials_model = train_model(industrials_sets)
real_estate_model = train_model(real_estate_sets)
technology_model = train_model(technology_sets)
materials_model = train_model(materials_sets)
utilities_model = train_model(utilities_sets)"""

In [ ]:
predict_overall = predict(overall_model, overall_sets, val_or_test='val')
"""predict_cm = predict(communication_model, communication_sets, val_or_test='val')
predict_e = predict(energy_model, energy_sets, val_or_test='val')
predict_cd = predict(consumer_discretionary_model, consumer_discretionary_sets, val_or_test='val')
predict_cs = predict(consumer_staples_model, consumer_staples_sets, val_or_test='val')
predict_f = predict(financials_model, financials_sets, val_or_test='val')
predict_h = predict(healthcare_model, healthcare_sets, val_or_test='val')
predict_i = predict(industrials_model, industrials_sets, val_or_test='val')
predict_re = predict(real_estate_model, real_estate_sets, val_or_test='val')
predict_t = predict(technology_model, technology_sets, val_or_test='val')
predict_m = predict(materials_model, materials_sets, val_or_test='val')
predict_u = predict(utilities_model, utilities_sets, val_or_test='val')"""

In [ ]:
predict_overall

In [ ]:
importance = overall_model.get_booster().get_score(importance_type='weight')  # or 'gain', 'cover'
importance2 = overall_model.get_booster().get_score(importance_type='gain')  # or 'gain', 'cover'
importance3 = overall_model.get_booster().get_score(importance_type='cover')  # or 'gain', 'cover'

print(importance2)

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

param_dist = {
    'n_estimators': [100, 200, 300, 400, 500, 700],
    'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6, 8, 10, 12],
    'min_child_weight': [1, 3, 5, 7, 10],
    'gamma': [0, 0.1, 0.25, 0.5, 1],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'reg_alpha': [0, 0.1, 0.5, 1, 5, 10],
    'reg_lambda': [0, 0.1, 1, 5, 10, 20],
    'booster': ['gbtree'],  # Could add 'dart' if you're interested in dropout boosting
    'tree_method': ['hist'],  # 'hist' for faster CPU training; use 'gpu_hist' if on GPU
    'objective': ['multi:softprob'],
    'num_class': [3],  # Set according to your number of classes
}

xgb_clf = XGBClassifier(
    use_label_encoder=False,
    random_state=42,
    verbosity=0
)

search = RandomizedSearchCV(
    estimator=xgb_clf,
    param_distributions=param_dist,
    n_iter=75,
    scoring='f1_macro',  # or 'accuracy', or 'roc_auc_ovr'
    cv=3,
    verbose=2,
    n_jobs=-1,
    random_state=42
)

#search.fit(X_train, y_train, sample_weight=sample_weights)
#print("Best Params:", search.best_params_)



In [ ]:
"""booster = model.get_booster()
explainer = shap.TreeExplainer(booster)
shap_values = explainer.shap_values(X_test)

# Initialize the SHAP JavaScript library
shap.initjs()"""

In [ ]:
"""#shap.summary_plot(shap_values, X_test, plot_type="bar")
#shap.summary_plot(shap_values[:, :, 0], X_test) # buy
#shap.summary_plot(shap_values[:, :, 1], X_test) # hold
shap.summary_plot(shap_values[:, :, 2], X_test) # sell
shap.dependence_plot("Bull_Probability", shap_values[:, :, 2], X_test)
shap.dependence_plot("ATR", shap_values[:, :, 2], X_test)
shap.dependence_plot("MACD", shap_values[:, :, 2], X_test)"""





In [ ]:
# Implementing with a real portfolio
real_stock_portfolio_1 = [
    "AMD", "NVDA", "RDDT", "SMCI"
]

real_stock_portfolio_2 = [
    "CRDO", "GOOG", "APP", "ASAN", "CLS", "META", "QQQ", "LUNR", "JPM", "PANW", "HOOD", "SPY", "SKYW", "TSM", "UBER", "VRT", "WMT", "NLR", "URA", "BITO", "COST", "XYZ"
]

real_stock_portfolio_3 = [
    "AAPL", "ABNB", "ACN", "ALAB", "AMD", "AMZN", "ANET", "AOSL", "APP",
    "ASAN", "ASML", "AVGO", "BAH", "BITO", "BWXT", "CLS", "COHR", "COIN", "COST",
    "COWG", "CPRX", "CRDO", "CRM", "CRWV", "DAVE", "DELL", "DKNG", "DOCS",
    "DXPE", "EPD", "FBTC", "FVRR", "GOOG", "GRNY", "HOOD", "IHAK", "INTA", "IONQ",
    "JPM", "LITE", "LQDT", "LUNR", "META", "MRVL", "MSFT", "MU", "NBIS", "NEE",
    "NFLX", "NLR", "NNE", "NUTX", "NVDA", "NVDY", "NVO", "OUST", "OXY", "PANW",
    "PEP", "PLD", "PLTR", "PYPL", "QCOM", "QTUM", "RBRK", "RDDT", "RDNT", "REAL",
    "RGTI", "S", "SAIC", "SCHD", "SEZL", "SKYW", "SMCI", "SMTC", "SNOW", "SOXL",
    "SYM", "TEAM", "TEM", "TOST", "TSM", "U", "UBER", "UPST", "URA", "VIST",
    "VRT", "WMT", "WRD", "XYZ", "HIMS", "OSCR"
]

In [ ]:
def run_model(stock_portfolio):
    # Part 1: Getting recommendations
    #download_and_fix_yfinance_data(stock_portfolio)
    real_df = create_stock_features(stock_portfolio, "data/stocks.csv")

    real_df = real_df.sort_values(by='Date')
    today_stocks_features = real_df.loc[[real_df.index.max()]]

    features = real_df.drop(['Stock', 'Returns-future-1wk', 'Returns-future-2wk'], axis=1).columns.values
    model = overall_model#XGBClassifier()

    """for t in stock_portfolio:
        info = yf.Ticker(t).info
        try:
            sector = info['sector']
        except KeyError:
            sector = 'default' # probably an etf or some other fund, not a stock

        match sector:
            case "Communication Services":
                model = communication_model
            case "Consumer Discretionary":
                model = consumer_discretionary_model
            case "Consumer Staples":
                model = consumer_staples_model
            case "Energy":
                model = energy_model
            case "Financials":
                model = financials_model
            case "Health Care":
                model = healthcare_model
            case "Industrials":
                model = industrials_model
            case "Technology":
                model = technology_model
            case "Materials":
                model = materials_model
            case "Real Estate":
                model = real_estate_model
            case "Utilities":
                model = utilities_model
            case _:
                model = overall_model"""

    todays_pred = model.predict(today_stocks_features[features])
    todays_probs = model.predict_proba(today_stocks_features[features])  # shape: (n_samples, 3)
    stock_col = today_stocks_features["Stock"].reset_index().drop(columns="Date")
    pred_col = pd.DataFrame(todays_pred)
    probs_col = pd.DataFrame(todays_probs)

    # Making recommendations per stock
    recommendations = stock_col.join(pred_col)
    recommendations.columns = ["Stock", "Recommendation"]
    recommendations["Recommendation"] = recommendations["Recommendation"].map({0: 'Sell', 1: 'Buy', 2: 'Hold'})

    #recommendations = stock_col.join(probs_col)
    #recommendations["Recommendation"] = recommendations["Recommendation"].map({0: 'Strong Sell', 1: 'Weak Sell', 2: 'Hold', 3: 'Weak Buy', 4: 'Strong Buy'})

    # Part 2: Getting the Markowitz mean-variance portfolio

    # Isolating buys
    """buy_recommendations = recommendations[recommendations.Recommendation == "Buy"].drop(columns="Recommendation")
    rec_array = buy_recommendations.to_numpy().flatten().tolist()
    buy_stocks_history = extract_ticker_dataframe("stocks.csv", rec_array[0])["Close"]
    for i in range(1, len(rec_array)):
        a = extract_ticker_dataframe("stocks.csv", rec_array[i])["Close"]
        buy_stocks_history = pd.concat([buy_stocks_history, a], axis=1, join='inner')
    buy_stocks_history.columns = rec_array

    def compute_adjusted_mu(buy_probs, baseline_mu, alpha=0.01):
        tickers = baseline_mu.index.intersection(buy_probs.index)
        adjusted_mu = baseline_mu.loc[tickers] * (1 + alpha * (buy_probs.loc[tickers] - 0.5))
        return adjusted_mu

    probs = model.predict_proba(today_stocks_features[features])
    probs_db = pd.DataFrame(probs, columns=["Hold", "Buy", "Sell"], index=today_stocks_features["Stock"].values)
    buy_probs = probs_db["Buy"]

    baseline_mu = expected_returns.mean_historical_return(buy_stocks_history)

    # Alpha is a strength parameter for the adjustment. Larger values will make the adjustment more aggressive.
    adjusted_mu = compute_adjusted_mu(buy_probs, baseline_mu, alpha=0.05)

    S = risk_models.sample_cov(buy_stocks_history)

    # Ensure that the adjusted_mu and S use the same tickers
    common_tickers = adjusted_mu.index.intersection(S.index)

    S = S.loc[common_tickers, common_tickers]
    ef = EfficientFrontier(adjusted_mu, S)
    ef.max_sharpe()
    clean_weights = ef.clean_weights()

    # Final portfolio weights
    weights = OrderedDict()
    for key, value in clean_weights.items():
        if value != 0.0:
            weights[key] = value"""
    return recommendations, 0
(recommendations, weights) = run_model(real_stock_portfolio_3) # <- Replace with the portfolio you want here



In [ ]:
#recommendations.sort_values("Recommendation", ascending=False, inplace=True)
recommendations.to_csv("recommendations.csv")

In [ ]:
print("Ideal Portfolio: \n")
print(weights)
